In [2]:
import scvelo as scv
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
import cellrank as cr
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

import psutil

mem = psutil.virtual_memory()
print(f"Total memory: {mem.total / 1e9:.2f} GB")
print(f"Available memory: {mem.available / 1e9:.2f} GB")
print(f"Used memory: {mem.used / 1e9:.2f} GB")
print(f"Memory usage: {mem.percent}%")
# print(globals().keys())

import gc
gc.collect()


Total memory: 405.11 GB
Available memory: 331.48 GB
Used memory: 70.28 GB
Memory usage: 18.2%


0

In [16]:
org_merge_changed = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_changed.h5ad")
org_merge_WT_changed = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_WT_changed.h5ad")
org_merge_KO_changed = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250726_D60_hg19/org_merge_KO_changed.h5ad")
# print(org_merge_WT_changed)

In [17]:
print(org_merge_changed)

AnnData object with n_obs × n_vars = 94531 × 1922
    obs: 'nCount_ambiguous', 'nFeature_ambiguous', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rps', 'percent.rpl', 'RNA_snn_res.0.1', 'seurat_clusters', 'pANN_0.25_0.3_354', 'doublet_finder', 'pANN_0.25_0.03_363', 'pANN_0.25_0.005_663', 'pANN_0.25_0.17_698', 'pANN_0.25_0.005_717', 'pANN_0.25_0.01_706', 'pANN_0.25_0.005_755', 'pANN_0.25_0.001_663', 'pANN_0.25_0.26_795', 'Sample_ID', 'unintegrated_clusters', 'clusters_CSS_allEmbeddingDims_Point1', 'clusters_CSS_SigPs_Point1', 'clusters_Harmony_Point1', 'clusters_CSS_allEmbeddingDims_Point2', 'clusters_CSS_SigPs_Point2', 'clusters_Harmony_Point2', 'clusters_CSS_allEmbeddingDims_Point5', 'clusters_CSS_SigPs_Point5', 'clusters_Harmony_Point5', 'clusters_CSS_SigPs_0.6', 'Genotype', 'Day', 'HNOCA_annot_level_1_pruned', 'HNOCA_annot_level_2_pruned', 'css_cluster', 'RNA_snn_res.0.8', 'initial_

In [58]:
# List the genes you want to check
genes_of_interest = ["DLX6"]  # replace with your genes

# Step 1: Find indices of the genes in adata.var
gene_indices = [org_merge_KO_changed.var.index[org_merge_KO_changed.var["gene_names"] == g][0] for g in genes_of_interest]
print(gene_indices)
# Step 2: Extract transcription (alpha), splicing (beta), degradation (gamma)
# These are stored in adata.var
transcription = org_merge_KO_changed_day20.var.loc[gene_indices, "fit_alpha"]
splicing      = org_merge_KO_changed_day20.var.loc[gene_indices, "fit_beta"]
degradation   = org_merge_KO_changed_day20.var.loc[gene_indices, "fit_gamma"]

# Step 3: Combine into a single table for easier viewing
import pandas as pd

df = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription (alpha)": transcription.values,
    "Splicing (beta)": splicing.values,
    "Degradation (gamma)": degradation.values
})

print(df)

gene_indices = [org_merge_WT_changed.var.index[org_merge_WT_changed.var["gene_names"] == g][0] for g in genes_of_interest]
print(gene_indices)
# Step 2: Extract transcription (alpha), splicing (beta), degradation (gamma)
# These are stored in adata.var
transcription = org_merge_WT_changed_day20.var.loc[gene_indices, "fit_alpha"]
splicing      = org_merge_WT_changed_day20.var.loc[gene_indices, "fit_beta"]
degradation   = org_merge_WT_changed_day20.var.loc[gene_indices, "fit_gamma"]

# Step 3: Combine into a single table for easier viewing
import pandas as pd

df1 = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription (alpha)": transcription.values,
    "Splicing (beta)": splicing.values,
    "Degradation (gamma)": degradation.values
})

print(df1)




['6760']
   Gene  Transcription (alpha)  Splicing (beta)  Degradation (gamma)
0  DLX6                    NaN              NaN                  NaN
['6760']


KeyError: "None of [Index(['6760'], dtype='object')] are in the [index]"

In [37]:
print(org_merge_WT_changed.var['fit_alpha'].max())

0.2672204525972922


In [69]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

# Combined dataframe (WT vs KO comparison)
df_compare = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription_WT": df1["Transcription (alpha)"].values,
    "Transcription_KO": df["Transcription (alpha)"].values,
    "Splicing_WT": df1["Splicing (beta)"].values,
    "Splicing_KO": df["Splicing (beta)"].values,
    "Degradation_WT": df1["Degradation (gamma)"].values,
    "Degradation_KO": df["Degradation (gamma)"].values
})

# Define colors
colors = {"WT": "#6AA94F", "KO": "#F1C232"}

# Use your original output path
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250902_gene_rates_WT_vs_KO_ventral.pdf"

rates = ["Transcription", "Splicing", "Degradation"]

with PdfPages(pdf_path) as pdf:
    for rate, wt_col, ko_col in zip(
        rates,
        ["Transcription_WT", "Splicing_WT", "Degradation_WT"],
        ["Transcription_KO", "Splicing_KO", "Degradation_KO"]
    ):
        # Copy and log-transform values
        df_plot = df_compare.copy()
        df_plot[[wt_col, ko_col]] = df_plot[[wt_col, ko_col]].applymap(lambda x: np.log1p(x))

        # Create bar plot
        fig, ax = plt.subplots(figsize=(10, 6))
        df_plot.plot(
            x="Gene",
            y=[wt_col, ko_col],
            kind="bar",
            ax=ax,
            color=[colors["WT"], colors["KO"]],
            width=0.7
        )

        ax.set_ylabel("log(1 + rate)")
        ax.set_title(f"{rate} rate (log scale)")
        ax.legend(["WT", "KO"], title="Genotype")

        # Save page to PDF
        pdf.savefig(fig)
        plt.close(fig)

print(f"Saved PDF with 3 plots to: {pdf_path}")


Saved PDF with 3 plots to: /srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250902_gene_rates_WT_vs_KO_ventral.pdf


In [4]:
print(org_merge_changed)

AnnData object with n_obs × n_vars = 94790 × 25077
    obs: 'nCount_ambiguous', 'nFeature_ambiguous', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rps', 'percent.rpl', 'RNA_snn_res.0.1', 'seurat_clusters', 'pANN_0.25_0.3_354', 'doublet_finder', 'pANN_0.25_0.03_363', 'pANN_0.25_0.005_663', 'pANN_0.25_0.17_698', 'pANN_0.25_0.005_717', 'pANN_0.25_0.01_706', 'pANN_0.25_0.005_755', 'pANN_0.25_0.001_663', 'pANN_0.25_0.26_795', 'Sample_ID', 'unintegrated_clusters', 'clusters_CSS_allEmbeddingDims_Point1', 'clusters_CSS_SigPs_Point1', 'clusters_Harmony_Point1', 'clusters_CSS_allEmbeddingDims_Point2', 'clusters_CSS_SigPs_Point2', 'clusters_Harmony_Point2', 'clusters_CSS_allEmbeddingDims_Point5', 'clusters_CSS_SigPs_Point5', 'clusters_Harmony_Point5', 'clusters_CSS_SigPs_0.6', 'Genotype', 'Day', 'HNOCA_annot_level_1_pruned', 'HNOCA_annot_level_2_pruned', 'css_cluster', 'RNA_snn_res.0.8'
    var: 

KeyError: 'fit_t_'

In [5]:
import pandas as pd

# Convert obs to a DataFrame
df = org_merge_changed.obs

# Find the index of the max latent time
max_idx = df["latent_time"].idxmax()

# Print the corresponding annotation and latent time
print("Cell ID:", max_idx)
print("Annotation:", df.loc[max_idx, "HNOCA_annot_level_2_pruned"])
print("Latent time:", df.loc[max_idx, "latent_time"])

Cell ID: D100.B8.org.KO52_TCATTACTCCTAAACG-D100B8KO52
Annotation: Ventral Telencephalic Neuron
Latent time: 1.0


In [66]:
scv.pp.filter_and_normalize(org_merge_WT_changed, min_cells = 2, min_cells_u = 2)
# scv.pp.filter_genes_dispersion(org_merge_WT_changed)
sc.pp.neighbors(org_merge_WT_changed, n_neighbors=20, use_rep='X_css')
scv.pp.moments(org_merge_WT_changed, n_pcs=None, n_neighbors=None)

Filtered out 3537 genes that are detected in less than 2 cells (spliced).
Filtered out 2791 genes that are detected in less than 2 cells (unspliced).
Normalized count data: X, spliced, unspliced.
Logarithmized X.


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


computing moments based on connectivities
    finished (0:00:54) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [67]:
scv.tl.recover_dynamics(org_merge_WT_changed, var_names=['7721', '1978', '1988'])

recovering dynamics (using 1/48 cores)
    finished (0:00:47) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)

outputs model fit of gene: 1988


In [5]:
scv.tl.velocity(org_merge_changed, mode='dynamical', diff_kinetics=True)
scv.tl.velocity_graph(org_merge_changed)

computing velocities
    finished (0:03:09) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/48 cores)
or disable the progress bar using `show_progress_bar=False`.
    finished (0:02:08) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [42]:
import pandas as pd

# mapping of cell types you want to keep
celltype_map = {
    "Neuroepithelium": "Neuroepithelium",
    "Dorsal Telencephalic IP": "Dorsal Telencephalic IP",
    "Dorsal Telencephalic Neuron": "Dorsal Telencephalic Neuron",
    "Ventral Telencephalic NPC": "Ventral Telencephalic NPC",
    "Ventral Telencephalic Neuron": "Ventral Telencephalic Neuron",
    "Non-telencephalic NPC": "Non-telencephalic NPC",
    "Non-telencephalic Neuron": "Non-telencephalic Neuron"
}

# create new obs column
org_merge_changed.obs["HNOCA_annot_level_2_pruned_simplified"] = (
    org_merge_changed.obs["HNOCA_annot_level_2_pruned"]  # your existing 14-celltype annotation
    .map(celltype_map)                       # map to simplified groups
    .fillna("Others")                        # everything else → Others
)

celltype_counts = org_merge_changed.obs["HNOCA_annot_level_2_pruned_simplified"].value_counts()

print("Number of cells per simplified cell type:")
print(celltype_counts)

Number of cells per simplified cell type:
HNOCA_annot_level_2_pruned_simplified
Ventral Telencephalic Neuron    36967
Non-telencephalic Neuron        29077
Others                          10683
Neuroepithelium                 10126
Non-telencephalic NPC            3930
Ventral Telencephalic NPC        3011
Dorsal Telencephalic Neuron       452
Dorsal Telencephalic IP           285
Name: count, dtype: int64


In [41]:
scv.pl.velocity_embedding_stream(org_merge_KO_changed, basis='umap.css.sigPCs', color=["HNOCA_annot_level_2_pruned_simplified"], legend_loc="lower left", save='/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/embedding_stream.pdf', show=False)

computing velocity embedding


KeyError: 'velocity'

In [51]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/org_merge_KO_changed_velocity_diff_kinetics_genotype_celltype_simplified_stream.pdf"

# r_palette = {
#     "Dorsal Telencephalic IP": "#E41A1C",
#     "Dorsal Telencephalic NPC": "#377EB8",
#     "MC": "#4DAF4A",
#     "NA": "#984EA3",
#     "NC Derivatives": "#FF7F00",
#     "Neuroepithelium": "#A65628",
#     "Non-telencephalic NPC": "#F781BF",
#     "Non-telencephalic Neuron": "#999999",
#     "PSC": "#66C2A5",
#     "Ventral Telencephalic NPC": "#FFD92F",
#     "Ventral Telencephalic Neuron": "#7F00FF",
#     "Astrocyte": "#4B0082",
#     "CP": "#056517",
#     "Dorsal Telencephalic Neuron": "#8DD3C7",
#     "Glioblast": "#FFA33F",
#     "OPC": "#80B1D3"
# }

r_palette = {
    "Neuroepithelium": "#f44336",
    "Dorsal Telencephalic IP": "#8bc34a",
    "Dorsal Telencephalic Neuron": "#009688",
    "Ventral Telencephalic NPC": "#1565c0",
    "Ventral Telencephalic Neuron": "#ffc107",
    "Non-telencephalic NPC": "#ff9800",
    "Non-telencephalic Neuron": "#448aff",
    "Others": "#ad1457"
}

with PdfPages(pdf_path) as pdf:

    scv.pl.velocity_embedding_grid(
        org_merge_KO_changed,
        basis='umap.css.sigPCs',
        color='HNOCA_annot_level_2_pruned_simplified',
        palette=r_palette,
        size=50,
        scale=0.3,               
        alpha=0.5,
        legend_loc='lower left',
        linewidth=1.2,
        legend_fontsize=5,
        show=False
    )

    
    fig = plt.gcf()
    fig.set_size_inches(11, 9)  
    pdf.savefig(fig)           
    plt.close()

In [10]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/org_merge_changed_latent_time_velocity_confidence.pdf"

scv.tl.latent_time(org_merge_changed)
scv.tl.velocity_confidence(org_merge_changed)

with PdfPages(pdf_path) as pdf:
    scv.pl.scatter(
        org_merge_changed,  
        color="latent_time", 
        color_map='gnuplot', 
        basis="X_umap.css.sigPCs",
        size=80,
        show=False
    )
    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    pdf.savefig(fig)
    plt.close()
    
    scv.pl.scatter(
        org_merge_changed, 
        color="velocity_length", 
        color_map="gnuplot", 
        basis="X_umap.css.sigPCs",
        size=80,
        show=False
    )
    fig = plt.gcf()
    fig.set_size_inches(10, 7)
    pdf.savefig(fig)
    plt.close()

    scv.pl.scatter(
        org_merge_changed, 
        color="velocity_confidence", 
        color_map="gnuplot", 
        basis="X_umap.css.sigPCs",
        size=80,
        show=False
    )

    fig = plt.gcf()
    fig.set_size_inches(10, 7)  
    pdf.savefig(fig)           
    plt.close()

computing latent time using root_cells as prior
    finished (0:00:42) --> added 
    'latent_time', shared time (adata.obs)
--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)


In [26]:
org_merge_changed.write("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/org_merge_changed.h5ad")

In [4]:
org_merge_changed.obs['condition_celltype'] = (
    org_merge_changed.obs['Genotype'].astype(str) + '_' +
    org_merge_changed.obs['HNOCA_annot_level_2_pruned'].astype(str)
).astype('category')
# scv.tl.differential_kinetic_test(org_merge_changed, groupby='condition_celltype')

In [ ]:
print(org_merge_changed.obs['HNOCA_annot_level_2_pruned'])

In [ ]:
var_df = org_merge_changed.var[['fit_diff_kinetics', 'fit_pval_kinetics']].copy()
var_df.index = org_merge_changed.var['gene_names']
var_df = var_df.sort_values(by='fit_pval_kinetics', ascending=True)
scv.get_df(var_df)
var_df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/fit_kinetics_results_sorted.csv")
top20_genes = var_df.head(20).index.tolist()
print(top20_genes)
top20_original_indices = org_merge_changed.var.index[org_merge_changed.var['gene_names'].isin(top20_genes)].tolist()
print(top20_original_indices)
top20_df = var_df.head(20)
top20_df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/top20_fit_kinetics_results.csv")

In [ ]:
r_palette_condition_celltype = {
    "WT_Dorsal Telencephalic IP": "#E41A1C",
    "WT_Dorsal Telencephalic NPC": "#377EB8",
    "WT_MC": "#4DAF4A",
    "WT_NA": "#984EA3",
    "WT_NC Derivatives": "#FF7F00",
    "WT_Neuroepithelium": "#A65628",
    "WT_Non-telencephalic NPC": "#F781BF",
    "WT_Non-telencephalic Neuron": "#999999",
    "WT_PSC": "#66C2A5",
    "WT_Ventral Telencephalic NPC": "#FFD92F",
    "WT_Ventral Telencephalic Neuron": "#7F00FF",
    "WT_Astrocyte": "#4B0082",
    "WT_CP": "#056517",
    "WT_Dorsal Telencephalic Neuron": "#8DD3C7",
    "WT_Glioblast": "#FFA33F",
    "WT_OPC": "#80B1D3",
    "KO_Dorsal Telencephalic IP": "#E41A1C",
    "KO_Dorsal Telencephalic NPC": "#377EB8",
    "KO_MC": "#4DAF4A",
    "KO_NA": "#984EA3",
    "KO_NC Derivatives": "#FF7F00",
    "KO_Neuroepithelium": "#A65628",
    "KO_Non-telencephalic NPC": "#F781BF",
    "KO_Non-telencephalic Neuron": "#999999",
    "KO_PSC": "#66C2A5",
    "KO_Ventral Telencephalic NPC": "#FFD92F",
    "KO_Ventral Telencephalic Neuron": "#7F00FF",
    "KO_Astrocyte": "#4B0082",
    "KO_CP": "#056517",
    "KO_Dorsal Telencephalic Neuron": "#8DD3C7",
    "KO_Glioblast": "#FFA33F",
    "KO_OPC": "#80B1D3"
}

for i, (gene, idx) in enumerate(zip(top20_genes, top20_original_indices), start=1):
    kwargs = dict(linewidth=2, add_linfit=False, frameon=False)
    scv.pl.scatter(
        org_merge_changed,
        idx,
        add_outline='fit_diff_kinetics',
        outline_width=(0.7, 0.05),  
        size=50,
        legend_loc='upper left', 
        palette=r_palette_condition_celltype,       
        legend_fontsize=6, 
        figsize=(10,10),
        **kwargs,
        save=None,
        show=False
    )
    
    plt.savefig(
        f"/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/{i}_{gene}.pdf"
    )
    plt.close()


In [ ]:
diff_clusters=list(org_merge_changed.var['fit_diff_kinetics'])
print(diff_clusters)
scv.pl.scatter(org_merge_changed, legend_loc='right', size=60, title='diff kinetics',
               add_outline=diff_clusters, outline_width=(.8, .2))

In [5]:
org_merge_changed.uns['neighbors']['distances'] = org_merge_changed.obsp['distances']
org_merge_changed.uns['neighbors']['connectivities'] = org_merge_changed.obsp['connectivities']

In [6]:
scv.tl.velocity_pseudotime(org_merge_changed)

In [7]:
scv.tl.paga(org_merge_changed, groups='HNOCA_annot_level_2_pruned')

running PAGA using priors: ['velocity_pseudotime']
    finished (0:00:10) --> added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns)
    'paga/transitions_confidence', velocity transitions (adata.uns)


In [55]:
df = scv.get_df(org_merge_changed, 'paga/transitions_confidence', precision=2).T

cluster_names = org_merge_changed.obs['HNOCA_annot_level_2_pruned'].cat.categories

df.index = cluster_names
df.columns = cluster_names

print(df)

                              Astrocyte   CP  Dorsal Telencephalic IP  \
Astrocyte                           0.0  0.0                      0.0   
CP                                  0.0  0.0                      0.0   
Dorsal Telencephalic IP             0.0  0.0                      0.0   
Dorsal Telencephalic NPC            0.0  0.0                      0.0   
Dorsal Telencephalic Neuron         0.0  0.0                      0.0   
Glioblast                           0.0  0.0                      0.0   
MC                                  0.0  0.0                      0.0   
NA                                  0.0  0.0                      0.0   
NC Derivatives                      0.0  0.0                      0.0   
Neuroepithelium                     0.0  0.0                      0.0   
Non-telencephalic NPC               0.0  0.0                      0.0   
Non-telencephalic Neuron            0.0  0.0                      0.0   
OPC                                 0.0  0.0       

In [48]:
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/org_merge_changed_paga.pdf"

with PdfPages(pdf_path) as pdf:
    scv.pl.paga(
        org_merge_changed, 
        alpha=0.1,
        size=50,
        legend_loc="upper left",
        legend_fontsize=6,
        basis="umap.css.sigPCs",
        threshold=0.05, 
        edge_width_scale=5,
        show=False
    )
    fig = plt.gcf()
    fig.set_size_inches(15, 10)
    pdf.savefig(fig)
    plt.close()

In [11]:
##BUFFER ZONE###

Defaulting to user installation because normal site-packages is not writeable
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 3.0.3
    Uninstalling Jinja2-3.0.3:
      Successfully uninstalled Jinja2-3.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygpcca 1.0.4 requires jinja2==3.0.3, but you have jinja2 3.1.6 which is incompatible.

[notice] A new release of pip available: 22.2.2 -> 25.2
[notice] To update, run: /apps/z_install_tree/linux-rocky8-ivybridge/gcc-12.2.0/python-3.10.8-pmtwsrrmcmrs6olvgx5xhepgh7gl5vro/bin/python3 -m pip install --upgrade pip


In [36]:
# Path to save PDF
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/org_merge_KO_changed_velocity.pdf"

# Convert to DataFrame for convenience
df = org_merge_KO_changed.obs.copy()


df = df[['velocity_length', 'Day']].dropna()
mean_latent = df.groupby('Day')['velocity_length'].mean()

# mean_latent = mean_latent.sort_values()

# Save plot into PDF
with PdfPages(pdf_path) as pdf:
    plt.figure(figsize=(8,5))
    mean_latent.plot(kind='bar')
    plt.ylabel("Average velocity length")
    plt.xlabel("Timepoints")
    plt.title("Average velocity length at different timepoints")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    pdf.savefig()   # saves the current figure into the PDF
    plt.close()     # close the figure so it doesn’t display twice


In [5]:
df = org_merge_changed.obs.copy()
df.to_csv("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/org_merge_combined_obs.csv", index=False)

In [75]:
# Count cells in the specified cell type
# a = sc.read("/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250630_css_only_combined_integration/PostIntegration_CSS_only_combined.h5ad")
num_dorsal_npc = (a.obs['HNOCA_annot_level_2_pruned'] 
                  == "Dorsal Telencephalic NPC").sum()

print(f"Number of Dorsal Telencephalic NPCs: {num_dorsal_npc}")


Number of Dorsal Telencephalic NPCs: 102


In [57]:
# List the genes you want to check
genes_of_interest = ["DLX6", "DLX1", "DLX2"]  # replace with your genes

# Step 1: Find indices of the genes in adata.var
gene_indices = [org_merge_WT_changed_day60.var.index[org_merge_WT_changed_day60.var["gene_names"] == g][0] for g in genes_of_interest]
print(gene_indices)
# Step 2: Extract transcription (alpha), splicing (beta), degradation (gamma)
# These are stored in adata.var
transcription = org_merge_KO_changed.var.loc[gene_indices, "fit_alpha"]
splicing      = org_merge_KO_changed.var.loc[gene_indices, "fit_beta"]
degradation   = org_merge_KO_changed.var.loc[gene_indices, "fit_gamma"]

# Step 3: Combine into a single table for easier viewing
import pandas as pd

df = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription (alpha)": transcription.values,
    "Splicing (beta)": splicing.values,
    "Degradation (gamma)": degradation.values
})

print(df)

gene_indices = [org_merge_WT_changed.var.index[org_merge_WT_changed.var["gene_names"] == g][0] for g in genes_of_interest]
print(gene_indices)
# Step 2: Extract transcription (alpha), splicing (beta), degradation (gamma)
# These are stored in adata.var
transcription = org_merge_WT_changed.var.loc[gene_indices, "fit_alpha"]
splicing      = org_merge_WT_changed.var.loc[gene_indices, "fit_beta"]
degradation   = org_merge_WT_changed.var.loc[gene_indices, "fit_gamma"]

# Step 3: Combine into a single table for easier viewing
import pandas as pd

df1 = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription (alpha)": transcription.values,
    "Splicing (beta)": splicing.values,
    "Degradation (gamma)": degradation.values
})

print(df1)




IndexError: index 0 is out of bounds for axis 0 with size 0

In [59]:
df = org_merge_changed.obs.copy()


org_merge_WT_changed_day20 = org_merge_WT_changed[org_merge_WT_changed.obs["Day"] == "20"].copy()
org_merge_KO_changed_day20 = org_merge_KO_changed[org_merge_KO_changed.obs["Day"] == "20"].copy()
org_merge_WT_changed_day60 = org_merge_WT_changed[org_merge_WT_changed.obs["Day"].isin(["20", "60"])].copy()
org_merge_KO_changed_day60 = org_merge_KO_changed[org_merge_KO_changed.obs["Day"].isin(["20", "60"])].copy()
org_merge_WT_changed_day100 = org_merge_WT_changed[org_merge_WT_changed.obs["Day"].isin(["20", "60", "100"])].copy()
org_merge_KO_changed_day100 = org_merge_KO_changed[org_merge_KO_changed.obs["Day"].isin(["20", "60", "100"])].copy()


# scv.pp.filter_and_normalize(org_merge_WT_changed, min_cells = 2, min_cells_u = 2)
# sc.pp.neighbors(org_merge_WT_changed, n_neighbors=20, use_rep='X_css')
# scv.pp.moments(org_merge_WT_changed, n_pcs=None, n_neighbors=None)
# ['7721', '1978', '1988']

In [60]:
var_names = ['6760', '1978', '1988']
org_merge_list = [org_merge_WT_changed_day20, org_merge_KO_changed_day20, org_merge_WT_changed_day60, org_merge_KO_changed_day60, org_merge_WT_changed_day100, org_merge_KO_changed_day100]

for org_merge in org_merge_list:
    scv.pp.filter_and_normalize(org_merge, min_cells = 2, min_cells_u = 2)
    sc.pp.neighbors(org_merge, n_neighbors=20, use_rep='X_css')
    scv.pp.moments(org_merge, n_pcs=None, n_neighbors=None)
    scv.tl.recover_dynamics(org_merge, var_names=var_names)

Filtered out 6224 genes that are detected in less than 2 cells (spliced).
Filtered out 2529 genes that are detected in less than 2 cells (unspliced).
Normalized count data: X, spliced, unspliced.
Logarithmized X.
computing moments based on connectivities


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


    finished (0:00:07) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
recovering dynamics (using 1/64 cores)
    finished (0:00:05) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)

outputs model fit of gene: 1988
Filtered out 4863 genes that are detected in less than 2 cells (spliced).
Filtered out 2753 genes that are detected in less than 2 cells (unspliced).
Normalized count data: X, spliced, unspliced.
Logarithmized X.


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


computing moments based on connectivities
    finished (0:00:09) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
recovering dynamics (using 1/64 cores)
    finished (0:00:06) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)

outputs model fit of gene: 1988
Filtered out 4843 genes that are detected in less than 2 cells (spliced).
Filtered out 2780 genes that are detected in less than 2 cells (unspliced).
Normalized count data: X, spliced, unspliced.
Logarithmized X.


...

computing moments based on connectivities
    finished (0:00:21) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
recovering dynamics (using 1/64 cores)
    finished (0:00:16) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)

outputs model fit of gene: 1988
Filtered out 3537 genes that are detected in less than 2 cells (spliced).
Filtered out 2791 genes that are detected in less than 2 cells (unspliced).
Normalized count data: X, spliced, unspliced.
Logarithmized X.


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


computing moments based on connectivities
    finished (0:00:50) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
recovering dynamics (using 1/64 cores)
    finished (0:00:40) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)

outputs model fit of gene: 1988
Filtered out 2915 genes that are detected in less than 2 cells (spliced).
Filtered out 2866 genes that are detected in less than 2 cells (unspliced).
Normalized count data: X, spliced, unspliced.
Logarithmized X.


/home/z5358666/.local/lib/python3.10/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)


computing moments based on connectivities
    finished (0:01:08) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
recovering dynamics (using 1/64 cores)
    finished (0:00:47) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)

outputs model fit of gene: 1988


In [62]:
import pandas as pd

# List of genes to check
genes_of_interest = ["DLX1", "DLX2", "DLX6"]

# ---------------------------
# KO data
# ---------------------------
alphas_ko, betas_ko, gammas_ko = [], [], []

for g in genes_of_interest:
    if g in org_merge_KO_changed_day20.var["gene_names"].values:
        idx = org_merge_KO_changed_day20.var.index[org_merge_KO_changed_day20.var["gene_names"] == g][0]
        alphas_ko.append(org_merge_KO_changed_day20.var.loc[idx, "fit_alpha"])
        betas_ko.append(org_merge_KO_changed_day20.var.loc[idx, "fit_beta"])
        gammas_ko.append(org_merge_KO_changed_day20.var.loc[idx, "fit_gamma"])
    else:
        alphas_ko.append(0)
        betas_ko.append(0)
        gammas_ko.append(0)

df = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription (alpha)": alphas_ko,
    "Splicing (beta)": betas_ko,
    "Degradation (gamma)": gammas_ko
})

print("KO:")
print(df)

# ---------------------------
# WT data
# ---------------------------
alphas_wt, betas_wt, gammas_wt = [], [], []

for g in genes_of_interest:
    if g in org_merge_WT_changed_day20.var["gene_names"].values:
        idx = org_merge_WT_changed_day20.var.index[org_merge_WT_changed_day20.var["gene_names"] == g][0]
        alphas_wt.append(org_merge_WT_changed_day20.var.loc[idx, "fit_alpha"])
        betas_wt.append(org_merge_WT_changed_day20.var.loc[idx, "fit_beta"])
        gammas_wt.append(org_merge_WT_changed_day20.var.loc[idx, "fit_gamma"])
    else:
        alphas_wt.append(0)
        betas_wt.append(0)
        gammas_wt.append(0)

df1 = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription (alpha)": alphas_wt,
    "Splicing (beta)": betas_wt,
    "Degradation (gamma)": gammas_wt
})

print("WT:")
print(df1)


KO:
   Gene  Transcription (alpha)  Splicing (beta)  Degradation (gamma)
0  DLX1              17.647642        34.584394             1.643886
1  DLX2               5.645717        44.940018             0.473779
2  DLX6               1.719596         5.370387             0.405836
WT:
   Gene  Transcription (alpha)  Splicing (beta)  Degradation (gamma)
0  DLX1               0.016094          0.49194             0.313056
1  DLX2               0.029012          0.61174             0.218682
2  DLX6               0.000000          0.00000             0.000000


In [63]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

# Combined dataframe (as you already have)
df_compare = pd.DataFrame({
    "Gene": genes_of_interest,
    "Transcription_WT": df1["Transcription (alpha)"].values,
    "Transcription_KO": df["Transcription (alpha)"].values,
    "Splicing_WT": df1["Splicing (beta)"].values,
    "Splicing_KO": df["Splicing (beta)"].values,
    "Degradation_WT": df1["Degradation (gamma)"].values,
    "Degradation_KO": df["Degradation (gamma)"].values
})

# Replace NaN with 0
df_compare = df_compare.fillna(0)

# Add splicing-to-degradation ratios
# Avoid division by zero
df_compare["SpliceDegRatio_WT"] = df_compare["Splicing_WT"] / df_compare["Degradation_WT"].replace(0, np.nan)
df_compare["SpliceDegRatio_KO"] = df_compare["Splicing_KO"] / df_compare["Degradation_KO"].replace(0, np.nan)
df_compare[["SpliceDegRatio_WT", "SpliceDegRatio_KO"]] = df_compare[["SpliceDegRatio_WT", "SpliceDegRatio_KO"]].fillna(0)

# Define colors
colors = {"WT": "#6aa84f", "KO": "#f1c232"}

# Output path
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/ventral_marker_transcription_splicing.pdf"

# Replace "Degradation" with "Splice/Degradation ratio"
rates = ["Transcription", "Splicing", "Splice/Degradation ratio"]

with PdfPages(pdf_path) as pdf:
    for rate, wt_col, ko_col in zip(
        rates, 
        ["Transcription_WT", "Splicing_WT", "SpliceDegRatio_WT"], 
        ["Transcription_KO", "Splicing_KO", "SpliceDegRatio_KO"]
    ):
        # Log-transform values (handle zeros safely)
        df_plot = df_compare.copy()
        df_plot[[wt_col, ko_col]] = df_plot[[wt_col, ko_col]].fillna(0).applymap(lambda x: np.log1p(x))
        
        # Create figure
        fig, ax = plt.subplots(figsize=(10,6))
        df_plot.plot(
            x="Gene",
            y=[wt_col, ko_col],
            kind="bar",
            ax=ax,
            color=[colors["WT"], colors["KO"]],
            width=0.7
        )
        ax.set_ylabel("log(1 + value)")
        ax.set_title(f"{rate} (log scale)")
        ax.legend(["WT", "KO"], title="Genotype")
        
        # Save page
        pdf.savefig(fig)
        plt.close(fig)

print(f"Saved PDF with 3 plots to: {pdf_path}")


Saved PDF with 3 plots to: /srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250702_seurat_no_day_60/ventral_marker_transcription_splicing.pdf


In [50]:
import pandas as pd
import numpy as np

# List of ventral marker genes
genes_of_interest = ["DLX3", "DLX1", "DLX2"]

def extract_rates(adata, genes):
    transcription_list = []
    splicing_list = []
    degradation_list = []

    for g in genes:
        # Check if gene exists
        if g in adata.var["gene_names"].values:
            idx = adata.var.index[adata.var["gene_names"] == g][0]
            transcription_list.append(adata.var.loc[idx, "fit_alpha"])
            splicing_list.append(adata.var.loc[idx, "fit_beta"])
            degradation_list.append(adata.var.loc[idx, "fit_gamma"])
        else:
            transcription_list.append(0)
            splicing_list.append(0)
            degradation_list.append(0)

    # Create DataFrame
    df = pd.DataFrame({
        "Gene": genes,
        "Transcription (alpha)": transcription_list,
        "Splicing (beta)": splicing_list,
        "Degradation (gamma)": degradation_list
    })

    # Add splicing-to-degradation ratio
    df["Splicing/Degradation"] = df["Splicing (beta)"] / df["Degradation (gamma)"].replace(0, np.nan)  # avoid division by 0

    # Replace NaN values with 0
    df = df.fillna(0)

    return df

# ------------------------
# Extract rates for each sample
# ------------------------

# KO day20
df_ko_day20 = extract_rates(org_merge_KO_changed_day20, genes_of_interest)
print("KO Day20:")
print(df_ko_day20)

# WT day20
df_wt_day20 = extract_rates(org_merge_WT_changed_day20, genes_of_interest)
print("WT Day20:")
print(df_wt_day20)

# KO day60
df_ko_day60 = extract_rates(org_merge_KO_changed_day60, genes_of_interest)
print("KO Day60:")
print(df_ko_day60)

# WT day60
df_wt_day60 = extract_rates(org_merge_WT_changed_day60, genes_of_interest)
print("WT Day60:")
print(df_wt_day60)

# KO day100
df_ko_day100 = extract_rates(org_merge_KO_changed_day100, genes_of_interest)
print("KO Day100:")
print(df_ko_day100)

# WT day100
df_wt_day100 = extract_rates(org_merge_WT_changed_day100, genes_of_interest)
print("WT Day100:")
print(df_wt_day100)


KO Day20:
   Gene  Transcription (alpha)  Splicing (beta)  Degradation (gamma)  \
0  DLX3               0.000000         0.000000             0.000000   
1  DLX1              17.647642        34.584394             1.643886   
2  DLX2               5.645717        44.940018             0.473779   

   Splicing/Degradation  
0              0.000000  
1             21.038191  
2             94.854362  
WT Day20:
   Gene  Transcription (alpha)  Splicing (beta)  Degradation (gamma)  \
0  DLX3               0.000000          0.00000             0.000000   
1  DLX1               0.016094          0.49194             0.313056   
2  DLX2               0.029012          0.61174             0.218682   

   Splicing/Degradation  
0              0.000000  
1              1.571414  
2              2.797400  
KO Day60:
   Gene  Transcription (alpha)  Splicing (beta)  Degradation (gamma)  \
0  DLX3               0.000000         0.000000             0.000000   
1  DLX1               1.906092         6

In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

# Define palette
palette = {"WT": "#6AA94F", "KO": "#F1C232"}

# Add Day and Genotype info to each dataframe
dfs = [
    (df_ko_day20, "KO", 20),
    (df_wt_day20, "WT", 20),
    (df_ko_day60, "KO", 60),
    (df_wt_day60, "WT", 60),
    (df_ko_day100, "KO", 100),
    (df_wt_day100, "WT", 100)
]

all_df = []
for d, geno, day in dfs:
    d = d.copy()
    d["Genotype"] = geno
    d["Day"] = day
    
    # Compute splicing-to-degradation ratio
    d["Transcription (alpha)"] = d["Transcription (alpha)"].replace(1e-8, np.nan)
    d["Splicing (beta)"] = d["Splicing (beta)"].replace(1e-8, np.nan)
    d["Degradation (gamma)"] = d["Degradation (gamma)"].replace(1e-8, np.nan)
    d["Splicing/Degradation"] = d["Splicing/Degradation"].replace(1e-8, 0)
    
    all_df.append(d)

all_df = pd.concat(all_df, ignore_index=True)
print(all_df)

# Reshape to long format
melted = all_df.melt(
    id_vars=["Gene", "Genotype", "Day"],
    value_vars=["Transcription (alpha)", "Splicing (beta)", "Splicing/Degradation"],
    var_name="Process",
    value_name="Value"
)

# Log-transform (avoid log(0) by adding small epsilon)
epsilon = 1e-8
melted["LogValue"] = np.log(melted["Value"] + epsilon)
print(melted["LogValue"])

# Save plots to PDF
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/aby_linegraph_WNT_splicing_ratio_ventral_misc.pdf"
with PdfPages(pdf_path) as pdf:
    genes = melted["Gene"].unique()
    processes = melted["Process"].unique()

    for gene in genes:
        for proc in processes:
            subset = melted[(melted["Gene"] == gene) & (melted["Process"] == proc)]

            plt.figure(figsize=(8,6))
            for genotype in ["WT", "KO"]:
                sub = subset[subset["Genotype"] == genotype]
                mean_vals = sub.groupby("Day")["LogValue"].mean().reindex([20, 60, 100])
                plt.plot(mean_vals.index, mean_vals.values, marker="o",
                         label=genotype, color=palette[genotype],
                         linestyle="-")

            plt.title(f"{gene} - {proc} across time")
            plt.xlabel("Day")
            plt.ylabel("Log Value")
            plt.xticks([20, 60, 100])
            plt.legend()
            pdf.savefig()
            plt.close()

print(f"PDF saved to {pdf_path}")


    Gene  Transcription (alpha)  Splicing (beta)  Degradation (gamma)  \
0   DLX1              17.647642        34.584394             1.643886   
1   DLX2               5.645717        44.940018             0.473779   
2   DLX6               1.719596         5.370387             0.405836   
3   DLX1               0.016094         0.491940             0.313056   
4   DLX2               0.029012         0.611740             0.218682   
5   DLX6               0.000000         0.000000             0.000000   
6   DLX1               1.906092         6.843265             0.516230   
7   DLX2               4.761101        40.776288             0.678587   
8   DLX6               0.608925         3.955442             0.391829   
9   DLX1               0.089603         3.135646             3.966244   
10  DLX2               0.024165         0.586088             0.361798   
11  DLX6               0.000000         0.000000             0.000000   
12  DLX1               2.085261         7.604060   

In [28]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import ranksums
import numpy as np
import pandas as pd

# Copy obs to a DataFrame
df = org_merge_changed.obs.copy()

# Replace Neuroepithelial cells with Neuroepithelium
df['HNOCA_annot_level_2_pruned_simplified'] = df['HNOCA_annot_level_2_pruned_simplified'].replace(
    {"Neuroepithelial cells": "Neuroepithelium"}
)

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250909_latent_time_summaryplots.pdf"

# Days to analyze
days = ['20', '60', '100']

# Define lineages
lineages = {
    "Dorsal": [
        "Dorsal Telencephalic IP",
        "Dorsal Telencephalic Neuron",
        "Dorsal Telencephalic NPC"
    ],
    "Ventral": [
        "Ventral Telencephalic NPC",
        "Ventral Telencephalic Neuron"
    ],
    "Non-telencephalic": [
        "Non-telencephalic NPC",
        "Non-telencephalic Neuron"
    ]
}

# Define subsets of days for analysis
day_subsets = {
    '20': ['20'],
    '60': ['20', '60'],
    '100': ['20', '60', '100']
}

# Color schemes
palette_genotype = {"WT": "#6AA94F", "KO": "#F1C232"}
palette_lineage = {
    "Neuroepithelium": "#caf0f8",
    "Dorsal": "#ffd6ff",
    "Ventral": "#00b4d8",
    "Non-telencephalic": "#9b5de5"
}
palette_celltypes = {
    "Neuroepithelium": "#0077b6",
    "Dorsal Telencephalic IP": "#0077b6",
    "Dorsal Telencephalic Neuron": "#0077b6",
    "Dorsal Telencephalic NPC": "#0077b6",
    "Ventral Telencephalic NPC": "#00b4d8",
    "Ventral Telencephalic Neuron": "#00b4d8",
    "Non-telencephalic NPC": "#90e0ef",
    "Non-telencephalic Neuron": "#90e0ef",
}

# PDF output
with PdfPages(pdf_path) as pdf:
    for day in days:
        subset_days = day_subsets[day]
        df_day = df[df['Day'].isin(subset_days)].copy()

        # Drop groups with ≤ 1 cell
        group_sizes = (
            df_day.groupby(['HNOCA_annot_level_2_pruned_simplified', 'Genotype'])
            .size()
            .reset_index(name='count')
        )
        valid_groups = group_sizes[group_sizes['count'] > 1][['HNOCA_annot_level_2_pruned_simplified', 'Genotype']]
        df_day = df_day.merge(valid_groups, on=['HNOCA_annot_level_2_pruned_simplified', 'Genotype'])

        # ===========================================================
        # 1. GENOTYPE-SPECIFIC LATENT TIME (Violin plot, sorted by mean)
        # ===========================================================
        genotype_means = df_day.groupby('Genotype')['latent_time'].mean().sort_values()
        sorted_genotype_order = genotype_means.index.tolist()

        fig, ax = plt.subplots(figsize=(6, 6))
        sns.violinplot(
            data=df_day,
            x="Genotype",
            y="latent_time",
            order=sorted_genotype_order,
            palette={g: palette_genotype[g] for g in sorted_genotype_order},
            inner=None,
            ax=ax
        )

        # Overlay mean points
        for i, genotype in enumerate(sorted_genotype_order):
            mean_val = df_day[df_day['Genotype'] == genotype]['latent_time'].mean()
            ax.plot(i, mean_val, 'o', color='black', markersize=5)

        # Wilcoxon rank-sum test
        if "WT" in sorted_genotype_order and "KO" in sorted_genotype_order:
            stat, pval = ranksums(
                df_day[df_day['Genotype'] == "WT"]['latent_time'],
                df_day[df_day['Genotype'] == "KO"]['latent_time']
            )
            y_max = df_day['latent_time'].max()
            ax.text(0.5, y_max * 1.05, f"Wilcoxon p = {pval:.3e}", ha='center', va='bottom', fontsize=12)

        ax.set_title(f"Day {day} - Genotype latent time average")
        pdf.savefig(fig)
        plt.close(fig)
        
        # ===========================================================
        # 2. LINEAGE AVERAGE LATENT TIME (Violin plot, sorted by mean)
        # ===========================================================
        df_day['Lineage'] = df_day['HNOCA_annot_level_2_pruned_simplified'].map(
            {ct: lineage for lineage, cells in lineages.items() for ct in cells}
        )
        df_day.loc[df_day['HNOCA_annot_level_2_pruned_simplified'] == "Neuroepithelium", "Lineage"] = "Neuroepithelium"
        df_lineage = df_day[df_day['Lineage'].notna()]
        
        lineage_means = df_lineage.groupby('Lineage')['latent_time'].mean().sort_values()
        sorted_lineage_order = lineage_means.index.tolist()
        
        fig, ax = plt.subplots(figsize=(8, 6))
        sns.violinplot(
            data=df_lineage,
            x="Lineage",
            y="latent_time",
            order=sorted_lineage_order,
            palette={l: palette_lineage[l] for l in sorted_lineage_order},
            inner=None,
            ax=ax
        )
        
        # Overlay mean points per lineage
        for i, lineage in enumerate(sorted_lineage_order):
            mean_val = df_lineage[df_lineage['Lineage'] == lineage]['latent_time'].mean()
            ax.plot(i, mean_val, 'o', color='black', markersize=5)
        
        ax.set_title(f"Day {day} - Lineage latent time average")
        pdf.savefig(fig)
        plt.close(fig)
        
        # ===========================================================
        # Wilcoxon rank-sum test between Ventral and Dorsal lineages
        # ===========================================================
        ventral_vals = df_lineage[df_lineage['Lineage'] == "Ventral"]['latent_time']
        dorsal_vals = df_lineage[df_lineage['Lineage'] == "Dorsal"]['latent_time']
        
        if len(ventral_vals) > 0 and len(dorsal_vals) > 0:
            stat, pval = ranksums(ventral_vals, dorsal_vals)
            print(f"Day {day} Ventral vs Dorsal Latent Time Wilcoxon p-value: {pval:.3e}")
        else:
            print(f"Day {day}: Not enough cells for Ventral vs Dorsal comparison")


        # ===========================================================
        # 3. CELL TYPE AVERAGE LATENT TIME (Violin plot, sorted)
        # ===========================================================
        df_celltypes = df_day[df_day['HNOCA_annot_level_2_pruned_simplified'] != "Others"].copy()
        cell_type_means = df_celltypes.groupby('HNOCA_annot_level_2_pruned_simplified')['latent_time'].mean()
        sorted_cell_types = cell_type_means.sort_values().index.tolist()

        fig, ax = plt.subplots(figsize=(14, 14))
        sns.violinplot(
            data=df_celltypes,
            x="HNOCA_annot_level_2_pruned_simplified",
            y="latent_time",
            order=sorted_cell_types,
            palette=palette_celltypes,
            inner=None,
            ax=ax
        )

        # Overlay mean points per cell type
        for i, cell_type in enumerate(sorted_cell_types):
            mean_val = df_celltypes[df_celltypes['HNOCA_annot_level_2_pruned_simplified'] == cell_type]['latent_time'].mean()
            ax.plot(i, mean_val, 'o', color='black', markersize=4)

        ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
        ax.set_title(f"Day {day} - Cell type latent time average (sorted)")
        pdf.savefig(fig)
        plt.close(fig)


Day 20: Not enough cells for Ventral vs Dorsal comparison


/scratch/pbs.6883669.kman.restech.unsw.edu.au/ipykernel_3918926/619926821.py:182: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")


Day 60 Ventral vs Dorsal Latent Time Wilcoxon p-value: 2.452e-13


/scratch/pbs.6883669.kman.restech.unsw.edu.au/ipykernel_3918926/619926821.py:182: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")


Day 100 Ventral vs Dorsal Latent Time Wilcoxon p-value: 1.346e-106


/scratch/pbs.6883669.kman.restech.unsw.edu.au/ipykernel_3918926/619926821.py:182: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")


In [25]:
print(df_lineage["Genotype"].unique())

['WT', 'KO']
Categories (2, object): ['KO', 'WT']


In [46]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import mannwhitneyu
import pandas as pd

# Copy obs to a DataFrame
df = org_merge_changed.obs.copy()

# Fix naming issue: "Neuroepithelial cells" → "Neuroepithelium"
df["HNOCA_annot_level_2_pruned_simplified"] = df["HNOCA_annot_level_2_pruned_simplified"].replace(
    {"Neuroepithelial cells": "Neuroepithelium"}
)


# Define lineage mapping
lineages = {
    "Dorsal": ["Dorsal Telencephalic NPC", "Dorsal Telencephalic Neuron", "Dorsal Telencephalic IP"],
    "Ventral": ["Ventral Telencephalic NPC", "Ventral Telencephalic Neuron", "Ventral Telencephalic IP"],
    "Non-telencephalic": ["Non-telencephalic NPC", "Non-telencephalic Neuron"],
}

# Assign lineages to each cell
df["Lineage"] = df["HNOCA_annot_level_2_pruned_simplified"].map(
    {ct: lineage for lineage, cells in lineages.items() for ct in cells}
)


# Keep only the three main lineages
df_lineage = df[df["Lineage"].notna()].copy()

# Normalize genotype labels
df_lineage["Genotype"] = df_lineage["Genotype"].astype(str).str.strip().str.upper()


# Palette for WT vs KO
palette = {"WT": "#6AA94F", "KO": "#F1C232"}

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250909_latent_time_violinplots/latent_time_WT_KO_lineages.pdf"

with PdfPages(pdf_path) as pdf:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    sns.violinplot(
        data=df_lineage,
        x="Lineage",
        y="latent_time",
        hue="Genotype",
        order=["Ventral", "Dorsal", "Non-telencephalic"],
        palette=palette,
        split=False,
        inner=None,
        ax=ax
    )

    # Set y limit padding so annotations have space
    y_max_all = df_lineage["latent_time"].max()
    ax.set_ylim(0, y_max_all * 1.2)

    # Add mean dots, counts, and Mann–Whitney p-values
    for i, lineage in enumerate(["Ventral", "Dorsal", "Non-telencephalic"]):
        sub = df_lineage[df_lineage["Lineage"] == lineage]

        if {"WT", "KO"}.issubset(sub["Genotype"].unique()):
            wt_vals = sub.loc[sub["Genotype"] == "WT", "latent_time"]
            ko_vals = sub.loc[sub["Genotype"] == "KO", "latent_time"]

            # Mean dots
            ax.plot(i - 0.2, wt_vals.mean(), "o", color="black", markersize=5)
            ax.plot(i + 0.2, ko_vals.mean(), "o", color="black", markersize=5)

            # Stats
            stat, p = mannwhitneyu(wt_vals, ko_vals, alternative="two-sided")

            # Annotate above violins (top of plot)
            ypos = y_max_all * 1.1
            ax.text(i, ypos, f"p = {p:.2e}",
                    ha="center", va="bottom", fontsize=10, fontweight="bold")

            # Add counts slightly below p-value
            ax.text(i - 0.2, ypos - 0.05 * y_max_all, f"n={len(wt_vals)}",
                    color=palette["WT"], ha="center", fontsize=9)
            ax.text(i + 0.2, ypos - 0.05 * y_max_all, f"n={len(ko_vals)}",
                    color=palette["KO"], ha="center", fontsize=9)

    ax.set_title("Latent time distribution by lineage and genotype")
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

print(f"PDF saved to {pdf_path}")


Number of cells per simplified cell type:
Lineage
Ventral              39978
Non-telencephalic    33007
Dorsal                 737
Name: count, dtype: int64
Number of cells per simplified cell type:
Genotype
KO    45283
WT    28439
Name: count, dtype: int64
PDF saved to /srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250909_latent_time_violinplots/latent_time_WT_KO_lineages.pdf


In [39]:
lineage_genotype_counts = df_lineage.groupby(["Lineage", "Genotype"]).size()
print("Number of cells per lineage and genotype:")
print(lineage_genotype_counts)

Number of cells per lineage and genotype:
Lineage            Genotype
Dorsal             KO             49
                   WT            688
Non-telencephalic  KO           7355
                   WT          25652
Ventral            KO          37879
                   WT           2099
dtype: int64


In [70]:
day = "100"
cell_type = "Dorsal Telencephalic NPC"

# Subset to KO only
df_day_KO = df[(df["Day"] == day) & (df["Genotype"] == "KO")]

# Get all rows for that cell type
df_subset = df_day_KO[df_day_KO["HNOCA_annot_level_2_pruned"] == cell_type]

if not df_subset.empty:
    print(f"All occurrences of '{cell_type}' in KO on Day {day}:")
    print(df_subset)
else:
    print(f"'{cell_type}' not found in KO on Day {day}.")


All occurrences of 'Dorsal Telencephalic NPC' in KO on Day 100:
                                              nCount_ambiguous  \
D100.B8.org.KO58_AATGGCTGTTAACCTG-D100B8KO58            1176.0   

                                              nFeature_ambiguous  \
D100.B8.org.KO58_AATGGCTGTTAACCTG-D100B8KO58                 630   

                                              nCount_spliced  \
D100.B8.org.KO58_AATGGCTGTTAACCTG-D100B8KO58          4528.0   

                                              nFeature_spliced  \
D100.B8.org.KO58_AATGGCTGTTAACCTG-D100B8KO58              2230   

                                              nCount_unspliced  \
D100.B8.org.KO58_AATGGCTGTTAACCTG-D100B8KO58            8582.0   

                                              nFeature_unspliced  \
D100.B8.org.KO58_AATGGCTGTTAACCTG-D100B8KO58                3099   

                                                    orig.ident  nCount_RNA  \
D100.B8.org.KO58_AATGGCTGTTAACCTG-D100B8KO58  D100.B8.KO

In [31]:
# root_cells_ko = org_merge_KO_changed.obs[
#     (org_merge_KO_changed.obs['HNOCA_annot_level_2_pruned'] == 'PSC')
# ].index.tolist()

# org_merge_KO_changed.obs['root_cells_manual'] = False
# org_merge_KO_changed.obs.loc[root_cells_ko, 'root_cells_manual'] = True

# # Recompute latent time using KO PSCs as root
# scv.tl.latent_time(org_merge_KO_changed, root_key='root_cells_manual')

# scv.pl.scatter(org_merge_KO_changed, color='latent_time', color_map='gnuplot', basis="umap.css.sigPCs", size=80)



# Path to save PDF
pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/org_merge_KO_changed_HNOCA_latent_time_PSC_set_root.pdf"

# Convert to DataFrame for convenience
df = org_merge_KO_changed.obs.copy()


df = df[['latent_time', 'HNOCA_annot_level_2_pruned']].dropna()
mean_latent = df.groupby('HNOCA_annot_level_2_pruned')['latent_time'].mean()

mean_latent = mean_latent.sort_values()

# Save plot into PDF
with PdfPages(pdf_path) as pdf:
    plt.figure(figsize=(8,5))
    mean_latent.plot(kind='bar')
    plt.ylabel("Average latent time")
    plt.xlabel("Cell type (HNOCA_annot_level_2_pruned)")
    plt.title("Average latent time per cell type (KO)")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    pdf.savefig()   # saves the current figure into the PDF
    plt.close()     # close the figure so it doesn’t display twice


In [70]:
import pandas as pd
from scipy.stats import mannwhitneyu

df_WT = org_merge_WT_changed.obs.copy()
df_KO = org_merge_KO_changed.obs.copy()

ventral_cells_WT = df_WT[df_WT['HNOCA_annot_level_2_pruned'].isin(['Ventral Telencephalic NPC', 'Ventral Telencephalic Neuron'])]
ventral_cells_KO = df_KO[df_KO['HNOCA_annot_level_2_pruned'].isin(['Ventral Telencephalic NPC', 'Ventral Telencephalic Neuron'])]

wt_ventral = ventral_cells_WT['latent_time']
ko_ventral = ventral_cells_KO['latent_time']

stat_ventral, pval_ventral = mannwhitneyu(wt_ventral, ko_ventral, alternative='two-sided')
mean_diff_ventral = ko_ventral.mean() - wt_ventral.mean()

print(f"P-value for latent time difference across ventral cell fates: {pval_ventral}")
print(f"Average difference (KO - WT): {mean_diff_ventral}")


#########
dorsal_cells_WT = df_WT[df_WT['HNOCA_annot_level_2_pruned'].isin(['Dorsal Telencephalic NPC', 'Dorsal Telencephalic IP', 'Dorsal Telencephalic Neuron'])]
dorsal_cells_KO = df_KO[df_KO['HNOCA_annot_level_2_pruned'].isin(['Dorsal Telencephalic NPC', 'Dorsal Telencephalic IP', 'Dorsal Telencephalic Neuron'])]


wt_dorsal = dorsal_cells_WT['latent_time']
ko_dorsal = dorsal_cells_KO['latent_time']

stat_dorsal, pval_dorsal = mannwhitneyu(wt_dorsal, ko_dorsal, alternative='two-sided')
mean_diff_dorsal = ko_dorsal.mean() - wt_dorsal.mean()

print(f"P-value for latent time difference across dorsal cell fates: {pval_dorsal}")
print(f"Average difference (KO - WT): {mean_diff_dorsal}")

KeyError: 'latent_time'

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages

pdf_path = "/srv/scratch/voineagu/PROJECTS/GavinLi/Combined/Results/250813_differential_kinetics/day100_distribution.pdf"

# --- Step 1: simplify labels ---
keep_labels = [
    "WT_Dorsal Telencephalic Neuron",
    "WT_Dorsal Telencephalic IP",
    "WT_Neuroepithelium",
    "WT_Non-telencephalic NPC",
    "WT_Non-telencephalic Neuron",
    "WT_Ventral Telencephalic NPC",
    "WT_Ventral Telencephalic Neuron",
    "KO_Dorsal Telencephalic Neuron",
    "KO_Dorsal Telencephalic IP",
    "KO_Neuroepithelium",
    "KO_Non-telencephalic NPC",
    "KO_Non-telencephalic Neuron",
    "KO_Ventral Telencephalic NPC",
    "KO_Ventral Telencephalic Neuron",
]

obs = org_merge_changed.obs.copy()
obs["condition_celltype"] = obs["condition_celltype"].astype(str)
obs["cond_celltype_simplified"] = np.where(
    obs["condition_celltype"].isin(keep_labels),
    obs["condition_celltype"],
    obs["condition_celltype"].str.split("_").str[0] + "_Others"
)

obs["Condition"] = obs["cond_celltype_simplified"].str.split("_").str[0]
obs["Celltype"] = obs["cond_celltype_simplified"].str.split("_").str[1]
obs.loc[obs["Celltype"] == "Others", "Celltype"] = "Others"

counts = obs.groupby(["Celltype", "Condition"]).size().reset_index(name="Count")
total_per_condition = obs.groupby("Condition").size()
counts["Fraction"] = counts.apply(lambda row: row["Count"] / total_per_condition[row["Condition"]], axis=1)

# --- Step 3: plotting ---
fig, ax = plt.subplots(figsize=(12, 6))
colors = {"WT": "#6AA94F", "KO": "#F1C232"}
celltypes_order = [
    "Neuroepithelium",
    "Dorsal Telencephalic IP",
    "Dorsal Telencephalic Neuron",
    "Ventral Telencephalic NPC",
    "Ventral Telencephalic Neuron",
    "Non-telencephalic NPC",
    "Non-telencephalic Neuron",
    "Others"
]
x = np.arange(len(celltypes_order))
bar_width = 0.35

for i, condition in enumerate(["WT", "KO"]):
    subdf = counts[counts["Condition"] == condition].set_index("Celltype").reindex(celltypes_order, fill_value=0)
    ax.bar(
        [pos + i*bar_width for pos in x],
        subdf["Fraction"],
        width=bar_width,
        label=condition,
        color=colors[condition]
    )

ax.set_xticks([pos + bar_width/2 for pos in x])
ax.set_xticklabels(celltypes_order, rotation=45, ha="right")
ax.set_ylabel("Fraction of Cells (log scale)")
ax.set_yscale("log")
ax.set_title("WT vs KO: Fraction of Each Cell Type (Including Others)")

# optional log ticks
ax.set_yticks([0.01, 0.1, 1.0])
ax.get_yaxis().set_major_formatter(plt.ScalarFormatter())
ax.get_yaxis().set_minor_formatter(plt.NullFormatter())
ax.set_yticklabels(["10⁻²", "10⁻¹", "10⁰"])

ax.legend(title="Condition")
plt.tight_layout()

# --- Step 4: Save using PdfPages ---
from matplotlib.backends.backend_pdf import PdfPages

with PdfPages(pdf_path) as pdf:
    plt.figure(figsize=(10,7))
    pdf.savefig(fig)  # saves the current figure to the PDF
    plt.close(fig)    # close the figure after saving


<Figure size 1000x700 with 0 Axes>